In [37]:
import pandas as pd
import os
from functools import reduce

from nlinec import get_models_dir, get_results_dir

In [38]:
ACCURACY_FILE = 'accuracy.csv'
MNLI_ACCURACY_FILE = 'mnli_accuracy.csv'

In [39]:
models = sorted(os.listdir(get_results_dir()))
print(models)

['nlinec-AE-2', 'nlinec-B-2', 'nlinec-C-2', 'nlinec-D-1', 'nlinec-D-2', 'nlinec-D-3', 'nlinec-E-2', 'roberta-large-mnli']


In [40]:
accuracy = {}
mnli_accuracy = {}
for model in models:
    if os.path.exists(os.path.join(get_results_dir(), model, ACCURACY_FILE)):
        accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

    if os.path.exists(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE)):
        mnli_accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

In [41]:
# Merge all accuracy dataframes on the 'granularity' column
accuracy_df = reduce(lambda left, right: pd.merge(left, right, on='granularity'), accuracy.values()).T
accuracy_df

,0,1,2
granularity,1.000000,2.000000,3.000000
nlinec-C-2,0.527657,0.588185,0.673861
nlinec-D-1,0.835219,0.360125,0.544365
nlinec-D-2,0.423691,0.588753,0.599520
nlinec-D-3,0.380962,0.499290,0.642686
nlinec-E-2,0.235643,0.046862,0.136691
roberta-large-mnli,0.460074,0.346492,0.441247


In [42]:
mnli_accuracy_df = pd.concat(mnli_accuracy.values(), axis=1).T
mnli_accuracy_df

,accuracy
nlinec-C-2,0.744880
nlinec-D-1,0.841875
nlinec-D-2,0.823128
nlinec-D-3,0.805502
nlinec-E-2,0.700662
roberta-large-mnli,0.902598


In [47]:
# Combine the accuracy and mnli_accuracy dataframes
# Rename the Granularity columns from 0, 1, 2 to 'Coarse', 'Medium', 'Fine'
# Rename the 'accuracy' column from the mnli_accuracy dataframe to 'MNLI Accuracy'
# Remove the 'granularity' row from the resulting dataframe
combined_df = pd.concat([accuracy_df, mnli_accuracy_df], axis=1).rename(columns={0: 'Coarse', 1: 'Medium', 2: 'Fine', 'accuracy': 'MNLI'}).drop('granularity', axis=0)
combined_df

,Coarse,Medium,Fine,MNLI
nlinec-C-2,0.527657,0.588185,0.673861,0.744880
nlinec-D-1,0.835219,0.360125,0.544365,0.841875
nlinec-D-2,0.423691,0.588753,0.599520,0.823128
nlinec-D-3,0.380962,0.499290,0.642686,0.805502
nlinec-E-2,0.235643,0.046862,0.136691,0.700662
roberta-large-mnli,0.460074,0.346492,0.441247,0.902598
